# 🚀 Adaptive 3D GNSS Degradation Model
## Multi-Dimensional Spatial Analysis with Auto-Feature Selection

**Author:** Sofia Buriak  
**Version:** 2.1 (Advanced Diploma)  
**Date:** January 2026

---

### 🎯 Mission Statement

This notebook implements a **3D Spatial Regression Model** that predicts degradation across three dimensions:

| Dimension | Target | Critical For |
|-----------|--------|-------------|
| **1D** | `vAcc` (Vertical) | Terrain collision avoidance |
| **2D** | `hAcc` (Horizontal) | Route navigation |
| **3D** | $\sqrt{hAcc^2 + vAcc^2}$ | Overall positioning |

### 💡 Key Innovations

1. **Auto-Correlation Analysis:** Automatically identifies optimal features for each dimension
2. **Physics-Based Engineering:** Signal energy, geometric stress, satellite efficiency
3. **Max-Based Target:** $Score = \max(Norm(hAcc), Norm(vAcc), Norm(3D))$
4. **Adaptive XGBoost:** Recall-optimized hyperparameters

---

In [ ]:
# ============================================================
# CELL 1: IMPORTS & CONFIGURATION
# ============================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import (
    mean_absolute_error, 
    mean_squared_error,
    precision_score, 
    recall_score, 
    f1_score,
    classification_report
)
import warnings
import gc
import os
import json

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# ========== CONFIGURATION ==========
CONFIG = {
    'DATA_PATH': '../data/processed/all_data_compressed.parquet',
    'MODEL_OUTPUT_PATH': '../models/gnss_adaptive_3d.json',
    'CONFIG_OUTPUT_PATH': '../models/gnss_adaptive_3d_config.json',
    
    # Target Engineering
    'SAFE_LIMIT_MM': 5000,      # 5 meters — Safe zone
    'FAIL_LIMIT_MM': 50000,     # 50 meters — Critical zone
    
    # Train/Test Split
    'TEST_START_DATE': '2025-12-01',
    
    # Model Hyperparameters (Diploma Verified)
    'XGB_PARAMS': {
        'n_estimators': 300,
        'max_depth': 7,
        'learning_rate': 0.03,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'objective': 'reg:logistic',
        'tree_method': 'hist',
        'n_jobs': -1,
        'random_state': 42
    },
    
    # Auto-Feature Selection
    'MIN_CORRELATION': 0.05
}

print("✅ Configuration loaded")
print(f"   Safe Limit: {CONFIG['SAFE_LIMIT_MM']/1000:.1f}m")
print(f"   Fail Limit: {CONFIG['FAIL_LIMIT_MM']/1000:.1f}m")

---
## 📂 Step 1: Data Loading
---

In [ ]:
# ============================================================
# CELL 2: DATA LOADING
# ============================================================
print("📂 LOADING DATA")
print("="*60)

try:
    df = pd.read_parquet(CONFIG['DATA_PATH'])
    print(f"   ✅ Loaded from parquet: {df.shape}")
except FileNotFoundError:
    print("   ⚠️ Parquet not found. Loading from raw CSV...")
    import sys
    sys.path.append('..')
    from src.data_loader import load_all_data
    df = load_all_data('../data/raw')

# Sort by time
df = df.sort_values('timestamp').reset_index(drop=True)

# Ensure datetime
if not np.issubdtype(df['timestamp'].dtype, np.datetime64):
    df['timestamp'] = pd.to_datetime(df['timestamp'])

# Memory optimization
float_cols = df.select_dtypes(include=['float64']).columns
df[float_cols] = df[float_cols].astype(np.float32)

print(f"\n📊 Dataset Summary:")
print(f"   Samples: {len(df):,}")
print(f"   Time range: {df['timestamp'].min()} → {df['timestamp'].max()}")
print(f"   Memory: {df.memory_usage(deep=True).sum() / 1e6:.1f} MB")

---
## 🔬 Step 2: Preliminary Data Intelligence

### Automatic Correlation Analysis:
1. Which factors most affect **1D error** (vAcc)? Hypothesis: `vDOP`, `cnoStd`
2. Which factors affect **2D error** (hAcc)? Hypothesis: `hDOP`, `numSV`
3. Auto-generate feature list, rejecting weak correlations (< 0.05)

---

In [ ]:
# ============================================================
# CELL 3: AUTOMATIC CORRELATION ANALYSIS
# ============================================================
print("🔬 PRELIMINARY DATA INTELLIGENCE")
print("="*60)

# Calculate 3D error
df['error_3d'] = np.sqrt(df['hAcc']**2 + df['vAcc']**2).astype(np.float32)

# Identify potential features
EXCLUDE_COLS = [
    'timestamp', 'hAcc', 'vAcc', 'sAcc', 'tAcc', 'error_3d',
    'overallPositionLabel', 'horizontalPositionLabel', 'verticalPositionLabel'
]

feature_cols = [col for col in df.columns 
                if col not in EXCLUDE_COLS 
                and df[col].dtype in ['float32', 'float64', 'int64', 'int32', 'int8', 'int16']]

print(f"\n📋 Candidate features: {len(feature_cols)}")

# Compute correlations
correlations = {
    '1D (vAcc)': {},
    '2D (hAcc)': {},
    '3D (Spatial)': {}
}

for feat in feature_cols:
    if df[feat].notna().sum() > 1000:
        correlations['1D (vAcc)'][feat] = df[feat].corr(df['vAcc'])
        correlations['2D (hAcc)'][feat] = df[feat].corr(df['hAcc'])
        correlations['3D (Spatial)'][feat] = df[feat].corr(df['error_3d'])

corr_df = pd.DataFrame(correlations).dropna()
corr_df = corr_df.reindex(corr_df['3D (Spatial)'].abs().sort_values(ascending=False).index)

print("\n📈 CORRELATION TABLE (sorted by 3D impact):")
print("-"*50)
display(corr_df.round(4))

In [ ]:
# ============================================================
# CELL 4: HYPOTHESIS VALIDATION
# ============================================================
print("\n🧪 HYPOTHESIS VALIDATION")
print("="*60)

# Hypothesis 1: vDOP and cnoStd affect vAcc
print("\n📐 Hypothesis 1: vDOP and cnoStd → vAcc (1D)")
if 'vDOP' in corr_df.index:
    vdop_corr = corr_df.loc['vDOP', '1D (vAcc)']
    print(f"   vDOP → vAcc:  r = {vdop_corr:.4f} {'✅ CONFIRMED' if abs(vdop_corr) > 0.1 else '⚠️ WEAK'}")
if 'cnoStd' in corr_df.index:
    cnostd_corr = corr_df.loc['cnoStd', '1D (vAcc)']
    print(f"   cnoStd → vAcc: r = {cnostd_corr:.4f} {'✅ CONFIRMED' if abs(cnostd_corr) > 0.1 else '⚠️ WEAK'}")

# Hypothesis 2: hDOP and numSV affect hAcc
print("\n📐 Hypothesis 2: hDOP and numSV → hAcc (2D)")
if 'hDOP' in corr_df.index:
    hdop_corr = corr_df.loc['hDOP', '2D (hAcc)']
    print(f"   hDOP → hAcc:  r = {hdop_corr:.4f} {'✅ CONFIRMED' if abs(hdop_corr) > 0.1 else '⚠️ WEAK'}")
if 'numSV' in corr_df.index:
    numsv_corr = corr_df.loc['numSV', '2D (hAcc)']
    print(f"   numSV → hAcc: r = {numsv_corr:.4f} {'✅ CONFIRMED' if abs(numsv_corr) > 0.05 else '⚠️ WEAK'}")

# Auto-select features
threshold = CONFIG['MIN_CORRELATION']
AUTO_SELECTED = corr_df[
    (corr_df['3D (Spatial)'].abs() >= threshold) | 
    (corr_df['1D (vAcc)'].abs() >= threshold) |
    (corr_df['2D (hAcc)'].abs() >= threshold)
].index.tolist()

print(f"\n✅ AUTO-SELECTED FEATURES ({len(AUTO_SELECTED)}):")
for i, feat in enumerate(AUTO_SELECTED, 1):
    print(f"   {i:2d}. {feat}")

In [ ]:
# ============================================================
# CELL 5: CORRELATION HEATMAP
# ============================================================
fig, ax = plt.subplots(figsize=(10, max(6, len(corr_df)*0.4)))

# Create heatmap
sns.heatmap(corr_df, annot=True, fmt='.3f', cmap='RdBu_r', 
            center=0, vmin=-1, vmax=1, ax=ax,
            linewidths=0.5, linecolor='white')

ax.set_title('🔬 Feature-Error Correlation Matrix', fontsize=14, fontweight='bold')
ax.set_xlabel('Error Dimension')
ax.set_ylabel('Feature')

plt.tight_layout()
plt.savefig('../figures/correlation_heatmap_3d.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 🛠️ Step 3: Advanced Feature Engineering

### Physics-Based Features:

| Feature | Formula | Physical Meaning |
|---------|---------|------------------|
| Signal Energy | `cnoMean × numSV` | Total constellation power |
| Sat Efficiency | `numSatsTracked / numSV` | Tracking quality |
| Geometric Stress | `vDOP × hDOP` | Combined poor geometry |
| CNO Derivative | `diff(cnoMean)` | Signal change rate |

---

In [ ]:
# ============================================================
# CELL 6: ADVANCED FEATURE ENGINEERING
# ============================================================
print("🛠️ ADVANCED FEATURE ENGINEERING")
print("="*60)

# Set timestamp as index for rolling operations
df = df.set_index('timestamp')

# ========== PHYSICS-BASED FEATURES ==========
print("\n📐 Creating physics-based features...")

# 1. Signal Energy
df['signal_energy'] = (df['cnoMean'] * df['numSV']).astype(np.float32)
print("   ✅ signal_energy = cnoMean × numSV")

# 2. Satellite Efficiency
df['sat_efficiency'] = (df['numSV'] / df['numSatsTracked'].replace(0, 1)).clip(0, 5).astype(np.float32)
print("   ✅ sat_efficiency = numSV / numSatsTracked")

# 3. Geometric Stress
if 'vDOP' in df.columns and 'hDOP' in df.columns:
    df['geometric_stress'] = (df['vDOP'] * df['hDOP']).astype(np.float32)
    print("   ✅ geometric_stress = vDOP × hDOP")

# 4. DOP Asymmetry
if 'vDOP' in df.columns and 'hDOP' in df.columns:
    df['dop_asymmetry'] = ((df['vDOP'] - df['hDOP']).abs()).astype(np.float32)
    print("   ✅ dop_asymmetry = |vDOP - hDOP|")

# ========== TEMPORAL DERIVATIVES ==========
print("\n⏱️ Creating temporal features...")

# CNO rate of change
df['cnoMean_diff'] = df['cnoMean'].diff().fillna(0).astype(np.float32)
print("   ✅ cnoMean_diff")

# Lag features
for col in ['cnoMean', 'sat_efficiency', 'numSV']:
    if col in df.columns:
        df[f'{col}_lag1'] = df[col].shift(1).bfill().astype(np.float32)
        df[f'{col}_lag5'] = df[col].shift(5).bfill().astype(np.float32)
print("   ✅ Lag features (t-1, t-5)")

# ========== ROLLING FEATURES (STABILITY) ==========
print("\n📊 Creating stability features...")

rolling_window = '10s'

# CNO stability
df['cnoMean_rolling_mean'] = df['cnoMean'].rolling(rolling_window).mean().astype(np.float32)
df['cnoMean_rolling_std'] = df['cnoMean'].rolling(rolling_window).std().fillna(0).astype(np.float32)
print(f"   ✅ cnoMean_rolling_std ({rolling_window})")

# vDOP stability
if 'vDOP' in df.columns:
    df['vDOP_rolling_std'] = df['vDOP'].rolling(rolling_window).std().fillna(0).astype(np.float32)
    print(f"   ✅ vDOP_rolling_std ({rolling_window})")

# numSV stability
df['numSV_rolling_std'] = df['numSV'].rolling(rolling_window).std().fillna(0).astype(np.float32)
print(f"   ✅ numSV_rolling_std ({rolling_window})")

# Reset index
df = df.reset_index()

print("\n✅ Feature engineering complete!")

---
## 🎯 Step 4: 3D Target Engineering

### Max-Based Degradation Score:

$$Score = \max\left( \frac{hAcc - Safe}{Fail - Safe}, \frac{vAcc - Safe}{Fail - Safe}, \frac{\sqrt{hAcc^2 + vAcc^2} - Safe}{Fail - Safe} \right)$$

**Rationale:** If the drone loses altitude (1D) but 2D is normal — Score still remains high (Alert).

---

In [ ]:
# ============================================================
# CELL 7: 3D TARGET ENGINEERING
# ============================================================
print("🎯 3D TARGET ENGINEERING")
print("="*60)

SAFE = CONFIG['SAFE_LIMIT_MM']
FAIL = CONFIG['FAIL_LIMIT_MM']

# Individual dimension scores
df['score_1d'] = ((df['vAcc'] - SAFE) / (FAIL - SAFE)).clip(0, 1).astype(np.float32)  # Altitude
df['score_2d'] = ((df['hAcc'] - SAFE) / (FAIL - SAFE)).clip(0, 1).astype(np.float32)  # Horizontal
df['score_3d'] = ((df['error_3d'] - SAFE) / (FAIL - SAFE)).clip(0, 1).astype(np.float32)  # Spatial

# MAX-BASED TARGET (Most conservative)
df['degradation_score'] = df[['score_1d', 'score_2d', 'score_3d']].max(axis=1).astype(np.float32)

print(f"\n📊 Target Formula: Score = max(1D, 2D, 3D)")
print(f"   Safe threshold: {SAFE/1000:.1f} m")
print(f"   Fail threshold: {FAIL/1000:.1f} m")

print(f"\n📈 Score Distribution:")
print(f"   Safe (< 0.1):     {(df['degradation_score'] < 0.1).sum():,} ({(df['degradation_score'] < 0.1).mean()*100:.1f}%)")
print(f"   Gray (0.1-0.9):   {((df['degradation_score'] >= 0.1) & (df['degradation_score'] <= 0.9)).sum():,}")
print(f"   Critical (> 0.9): {(df['degradation_score'] > 0.9).sum():,} ({(df['degradation_score'] > 0.9).mean()*100:.1f}%)")

In [ ]:
# ============================================================
# CELL 8: DIMENSIONAL SCORE COMPARISON
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Sample for visualization
sample = df.sample(min(50000, len(df)), random_state=42)

# 1D vs 2D
axes[0].hexbin(sample['score_1d'], sample['score_2d'], 
               gridsize=50, cmap='YlOrRd', mincnt=1)
axes[0].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[0].set_xlabel('1D Score (Altitude)')
axes[0].set_ylabel('2D Score (Horizontal)')
axes[0].set_title('1D vs 2D Score')

# 2D vs 3D
axes[1].hexbin(sample['score_2d'], sample['score_3d'], 
               gridsize=50, cmap='YlGnBu', mincnt=1)
axes[1].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[1].set_xlabel('2D Score (Horizontal)')
axes[1].set_ylabel('3D Score (Spatial)')
axes[1].set_title('2D vs 3D Score')

# Final Score Distribution
axes[2].hist(sample['degradation_score'], bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[2].axvline(0.5, color='red', linestyle='--', label='Threshold')
axes[2].set_xlabel('Final Score (Max)')
axes[2].set_ylabel('Count')
axes[2].set_title('Final Max Score Distribution')
axes[2].legend()

plt.suptitle('🎯 Multi-Dimensional Score Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/dimensional_scores.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 📋 Step 5: Feature Selection
---

In [ ]:
# ============================================================
# CELL 9: FEATURE SELECTION
# ============================================================
print("📋 FINAL FEATURE SELECTION")
print("="*60)

# Forbidden (target-related)
FORBIDDEN = [
    'timestamp', 'hAcc', 'vAcc', 'sAcc', 'tAcc', 'error_3d',
    'score_1d', 'score_2d', 'score_3d', 'degradation_score',
    'overallPositionLabel', 'horizontalPositionLabel', 'verticalPositionLabel'
]

# Build feature list
FEATURE_GROUPS = {
    'Core Signal': ['cnoMean', 'cnoStd', 'cnoMin', 'cnoMax'],
    'Satellites': ['numSV', 'numSatsTracked'],
    'Geometry (DOP)': ['vDOP', 'hDOP', 'pDOP', 'gDOP', 'nDOP', 'eDOP', 'tDOP'],
    'Engineered': ['signal_energy', 'sat_efficiency', 'geometric_stress', 'dop_asymmetry', 'cnoMean_diff'],
    'Stability': ['cnoMean_rolling_std', 'cnoMean_rolling_mean', 'vDOP_rolling_std', 'numSV_rolling_std'],
    'Temporal': [c for c in df.columns if 'lag' in c]
}

FINAL_FEATURES = []
for group, features in FEATURE_GROUPS.items():
    valid = [f for f in features if f in df.columns and f not in FORBIDDEN]
    FINAL_FEATURES.extend(valid)
    print(f"   {group}: {len(valid)} features")

# Remove duplicates
FINAL_FEATURES = list(dict.fromkeys(FINAL_FEATURES))

print(f"\n✅ TOTAL FEATURES: {len(FINAL_FEATURES)}")

# Fill NaN
for col in FINAL_FEATURES:
    df[col] = df[col].fillna(0)

---
## ✂️ Step 6: Train/Test Split
---

In [ ]:
# ============================================================
# CELL 10: TRAIN/TEST SPLIT
# ============================================================
print("✂️ TEMPORAL TRAIN/TEST SPLIT")
print("="*60)

split_date = pd.Timestamp(CONFIG['TEST_START_DATE'])

train_mask = df['timestamp'] < split_date
test_mask = df['timestamp'] >= split_date

X_train = df.loc[train_mask, FINAL_FEATURES].copy()
y_train = df.loc[train_mask, 'degradation_score'].copy()

X_test = df.loc[test_mask, FINAL_FEATURES].copy()
y_test = df.loc[test_mask, 'degradation_score'].copy()

# Store for later analysis
test_df = df[test_mask].copy()

print(f"   Train: {len(X_train):,} (before {split_date.date()})")
print(f"   Test:  {len(X_test):,} (from {split_date.date()})")
print(f"   Train Attack Rate: {(y_train > 0.5).mean()*100:.2f}%")
print(f"   Test Attack Rate:  {(y_test > 0.5).mean()*100:.2f}%")

gc.collect()

---
## 🤖 Step 7: Adaptive XGBoost Training
---

In [ ]:
# ============================================================
# CELL 11: MODEL TRAINING
# ============================================================
print("🤖 TRAINING ADAPTIVE XGBOOST")
print("="*60)

print("\n⚙️ Hyperparameters:")
for k, v in CONFIG['XGB_PARAMS'].items():
    print(f"   {k}: {v}")

model = XGBRegressor(**CONFIG['XGB_PARAMS'])

print("\n🏋️ Training...")
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=50
)

print("\n✅ Training complete!")

In [ ]:
# ============================================================
# CELL 12: FEATURE IMPORTANCE
# ============================================================
importance_df = pd.DataFrame({
    'feature': FINAL_FEATURES,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=True)

plt.figure(figsize=(10, max(6, len(importance_df)*0.35)))
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(importance_df)))
plt.barh(importance_df['feature'], importance_df['importance'], color=colors)
plt.xlabel('Importance')
plt.title('🔬 Feature Importance (XGBoost)', fontsize=14, fontweight='bold')
plt.grid(True, axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/feature_importance_adaptive.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n🏆 TOP 5 FEATURES:")
for _, row in importance_df.tail(5).iloc[::-1].iterrows():
    print(f"   {row['feature']:30s} {row['importance']:.4f}")

---
## 📈 Step 8: Multi-Dimensional Validation
---

In [ ]:
# ============================================================
# CELL 13: PREDICTIONS
# ============================================================
print("🔮 GENERATING PREDICTIONS")
print("="*60)

y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)
y_true_binary = (y_test > 0.5).astype(int)

print(f"   Predictions generated for {len(y_pred):,} samples")

In [ ]:
# ============================================================
# CELL 14: MULTI-DIMENSIONAL METRICS
# ============================================================
print("\n📈 MULTI-DIMENSIONAL VALIDATION")
print("="*60)

# Overall regression metrics
mae_total = mean_absolute_error(y_test, y_pred)
rmse_total = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\n🎯 OVERALL REGRESSION:")
print(f"   MAE:  {mae_total:.4f}")
print(f"   RMSE: {rmse_total:.4f}")

# Dimensional MAE
print(f"\n📐 DIMENSIONAL MAE:")
mae_1d = mean_absolute_error(test_df['score_1d'], y_pred)
mae_2d = mean_absolute_error(test_df['score_2d'], y_pred)
mae_3d = mean_absolute_error(test_df['score_3d'], y_pred)

print(f"   Metric 1D (Altitude/vAcc):   MAE = {mae_1d:.4f}")
print(f"   Metric 2D (Horizontal/hAcc): MAE = {mae_2d:.4f}")
print(f"   Metric 3D (Spatial):         MAE = {mae_3d:.4f}")

# Classification metrics
print(f"\n🎯 CLASSIFICATION (Threshold 0.5):")
recall = recall_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary, zero_division=0)
f1 = f1_score(y_true_binary, y_pred_binary)

print(f"   Precision: {precision:.4f}")
print(f"   Recall:    {recall:.4f}")
print(f"   F1 Score:  {f1:.4f}")

In [ ]:
# ============================================================
# CELL 15: CLASSIFICATION REPORT
# ============================================================
print("\n📊 DETAILED CLASSIFICATION REPORT:")
print(classification_report(y_true_binary, y_pred_binary, 
                           target_names=['Safe', 'Attack']))

In [ ]:
# ============================================================
# CELL 16: PREDICTION VS REALITY PLOT
# ============================================================
print("📊 VISUALIZATION: Prediction vs Reality")

# Find hardest case (attack period)
attack_indices = test_df[test_df['degradation_score'] > 0.5].index
if len(attack_indices) > 100:
    center_idx = attack_indices[len(attack_indices)//2]
    plot_start = max(test_df.index[0], center_idx - 300)
    plot_end = min(test_df.index[-1], center_idx + 300)
    
    plot_df = test_df.loc[plot_start:plot_end].copy()
    plot_pred = y_pred[plot_df.index - test_df.index[0]]
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Top: Scores
    ax1 = axes[0]
    ax1.plot(plot_df['timestamp'], plot_df['degradation_score'], 
             color='blue', alpha=0.6, label='Actual Score (Max 1D/2D/3D)')
    ax1.plot(plot_df['timestamp'], plot_pred, 
             color='red', linewidth=2, label='Predicted Score')
    ax1.axhline(0.5, color='gray', linestyle=':', label='Threshold')
    ax1.fill_between(plot_df['timestamp'], 0, plot_df['degradation_score'], 
                     where=plot_df['degradation_score'] > 0.5, 
                     color='red', alpha=0.2, label='Attack Zone')
    ax1.set_ylabel('Score')
    ax1.set_title('🎯 Prediction vs Reality (Hardest Case)', fontweight='bold')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1.05)
    
    # Bottom: Individual dimensions
    ax2 = axes[1]
    ax2.plot(plot_df['timestamp'], plot_df['score_1d'], 
             color='green', alpha=0.7, label='1D (Altitude)')
    ax2.plot(plot_df['timestamp'], plot_df['score_2d'], 
             color='orange', alpha=0.7, label='2D (Horizontal)')
    ax2.plot(plot_df['timestamp'], plot_df['score_3d'], 
             color='purple', alpha=0.7, label='3D (Spatial)')
    ax2.axhline(0.5, color='gray', linestyle=':')
    ax2.set_ylabel('Score')
    ax2.set_xlabel('Time')
    ax2.set_title('📐 Individual Dimension Scores', fontweight='bold')
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 1.05)
    
    plt.tight_layout()
    plt.savefig('../figures/prediction_vs_reality_adaptive.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("⚠️ Not enough attack samples for visualization")

---
## 💾 Step 9: Save Model
---

In [ ]:
# ============================================================
# CELL 17: SAVE MODEL & CONFIG
# ============================================================
print("💾 SAVING MODEL & CONFIGURATION")
print("="*60)

# Create directory
os.makedirs(os.path.dirname(CONFIG['MODEL_OUTPUT_PATH']), exist_ok=True)

# Save model
model.save_model(CONFIG['MODEL_OUTPUT_PATH'])
print(f"   ✅ Model: {CONFIG['MODEL_OUTPUT_PATH']}")

# Save config
model_config = {
    'model_name': 'gnss_adaptive_3d',
    'version': '2.1.0',
    'author': 'Sofia Buriak',
    'description': 'Adaptive 3D GNSS Degradation Model with Auto-Feature Selection',
    'input_features': FINAL_FEATURES,
    'hyperparameters': CONFIG['XGB_PARAMS'],
    'target_engineering': {
        'safe_limit_mm': CONFIG['SAFE_LIMIT_MM'],
        'fail_limit_mm': CONFIG['FAIL_LIMIT_MM'],
        'method': 'max(1D, 2D, 3D)'
    },
    'metrics': {
        'mae': float(mae_total),
        'rmse': float(rmse_total),
        'mae_1d': float(mae_1d),
        'mae_2d': float(mae_2d),
        'mae_3d': float(mae_3d),
        'recall': float(recall),
        'precision': float(precision),
        'f1': float(f1)
    }
}

with open(CONFIG['CONFIG_OUTPUT_PATH'], 'w') as f:
    json.dump(model_config, f, indent=4)
print(f"   ✅ Config: {CONFIG['CONFIG_OUTPUT_PATH']}")

In [ ]:
# ============================================================
# CELL 18: FINAL SUMMARY
# ============================================================
print("\n")
print("="*70)
print("🏆 FINAL SUMMARY: Adaptive 3D GNSS Model")
print("="*70)

print(f"""
📊 MODEL ARCHITECTURE:
   • Algorithm:      XGBoost Regressor (reg:logistic)
   • Features:       {len(FINAL_FEATURES)} auto-selected features
   • Target:         Max(1D, 2D, 3D) Degradation Score
   • Estimators:     {CONFIG['XGB_PARAMS']['n_estimators']}

📈 MULTI-DIMENSIONAL METRICS:
   ┌────────────────────┬──────────┐
   │ Dimension          │ MAE      │
   ├────────────────────┼──────────┤
   │ 1D (Altitude)      │ {mae_1d:.4f}   │
   │ 2D (Horizontal)    │ {mae_2d:.4f}   │
   │ 3D (Spatial)       │ {mae_3d:.4f}   │
   │ Overall            │ {mae_total:.4f}   │
   └────────────────────┴──────────┘

🎯 CLASSIFICATION (Attack Detection):
   • Precision: {precision:.4f}
   • Recall:    {recall:.4f}
   • F1 Score:  {f1:.4f}

💾 SAVED TO:
   • Model:  {CONFIG['MODEL_OUTPUT_PATH']}
   • Config: {CONFIG['CONFIG_OUTPUT_PATH']}
""")

print("="*70)
print("✅ MODEL READY FOR DEPLOYMENT!")
print("="*70)